In [1]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math
teams = pd.read_csv('teams_table.csv')
year1617 = pd.read_csv('../GamesData/Strength of Schedule/season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('../GamesData/Strength of Schedule/season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('../GamesData/Strength of Schedule/season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('../GamesData/Strength of Schedule/season1920.csv', parse_dates=['Date'])
curr_metrics = pd.read_csv('1-30-teammetrics.csv')
curr_metrics.dropna(inplace=True)
curr_season = pd.read_csv('2-1games.csv',parse_dates=['Date'])
curr_season['WinnerHome'] = np.where(curr_season['Winner']==curr_season['Home'],1,0)

In [2]:
frames = [year1617,year1718,year1819,year1920]
allyears = pd.concat(frames).drop(columns=['Unnamed: 0'])
allyears = allyears.drop_duplicates(subset=['Date', 'Home'],keep='last')
# full = allyears[['Home','Away','WinnerHome','KenPomVal_Away','BPIval_Away','SRS_Away','KenPomVal_Home','BPIval_Home','SRS_Home']]


In [3]:
curr_joined = curr_season.merge(curr_metrics,left_on=['Winner'],right_on=['teamID'],how='left')
curr_joined = curr_joined.merge(curr_metrics,left_on=['Loser'],right_on=['teamID'],how='left',suffixes=('_winner','_loser'))

In [4]:
curr_joined

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome,teamID_winner,kp_winner,...,ppg_loser,papg_loser,pace_loser,three_par_loser,ts_loser,tur_perc_loser,opp_three_par_loser,opp_ts_loser,opp_tur_perc_loser,date_loser
0,2020-11-25,335,76,27,67,27,335,0,335.0,26.09,...,72.7,78.2,71.9,0.464,0.539,17.2,0.352,0.568,16.5,2021-01-30 08:21:04
1,2020-11-25,336,89,302,54,336,302,1,336.0,25.71,...,67.5,75.6,68.5,0.282,0.502,17.3,0.442,0.541,13.0,2021-01-30 08:21:04
2,2020-11-25,128,97,204,67,128,204,1,128.0,27.09,...,76.8,77.4,73.5,0.324,0.561,17.4,0.333,0.571,21.5,2021-01-30 08:21:04
3,2020-11-25,351,77,79,67,351,79,1,351.0,23.79,...,69.5,75.0,70.6,0.341,0.498,16.2,0.359,0.554,17.3,2021-01-30 08:21:04
4,2020-11-25,121,122,203,60,121,203,1,121.0,25.52,...,71.9,78.9,75.0,0.340,0.506,16.5,0.414,0.558,20.2,2021-01-30 08:21:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,2021-01-31,281,75,160,73,160,281,0,281.0,10.64,...,71.7,69.6,68.3,0.375,0.547,17.6,0.390,0.505,13.2,2021-01-30 08:21:04
2321,2021-01-31,304,81,289,64,289,304,0,304.0,-4.16,...,64.4,67.6,68.0,0.397,0.500,17.2,0.371,0.505,14.1,2021-01-30 08:21:04
2322,2021-01-31,84,70,332,52,84,332,1,84.0,-9.54,...,69.0,70.8,71.2,0.417,0.509,16.1,0.352,0.524,18.0,2021-01-30 08:21:04
2323,2021-01-31,341,77,340,62,340,341,0,341.0,3.18,...,67.4,77.3,70.5,0.378,0.509,16.8,0.320,0.532,14.0,2021-01-30 08:21:04


### curr_season is the games that have happened in the 20-21 season
### curr_metrics is the metrics of the teams in the 20-21 season to join curr_season with
### allyears is the 2016-2020 seasons already joined with their metrics

#### 

#### Comparing Win Percentage of home team

In [5]:
print("2020-21 Season")
print("Number of Games won by home team:",curr_season['WinnerHome'].sum())
print("Number of Games played:",len(curr_season))
print("Home Win Percentage:","{:.0%}".format(curr_season['WinnerHome'].sum()/len(curr_season)))

2020-21 Season
Number of Games won by home team: 1451
Number of Games played: 2325
Home Win Percentage: 62%


In [6]:
print("2016-20 Seasons")
print("Number of Games won by home team:",allyears['WinnerHome'].sum())
print("Number of Games played:",len(allyears))
print("Home Win Percentage:","{:.0%}".format(allyears['WinnerHome'].sum()/len(allyears)))

2016-20 Seasons
Number of Games won by home team: 14178
Number of Games played: 21979
Home Win Percentage: 65%


#### Comparing Point Differential

In [7]:
home_wins_curr = curr_season[curr_season['WinnerHome'] == 1]
home_wins_curr['pt_diff'] = home_wins_curr['WinnerScore'] - home_wins_curr['LoserScore']
home_loss_curr = curr_season[curr_season['WinnerHome'] == 0]
home_loss_curr['pt_diff'] = home_loss_curr['WinnerScore'] - home_loss_curr['LoserScore']

c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
print('Point differential in Home Wins and Home Losses 20-21:', round(home_wins_curr['pt_diff'].mean(),2), round(home_loss_curr['pt_diff'].mean(),2))

Point differential in Home Wins and Home Losses 20-21: 14.09 10.5


In [9]:
home_wins_old = allyears[allyears['WinnerHome']==1]
home_wins_old['pt_diff'] = home_wins_old['Win_Score'] - home_wins_old['Lose_Score']
home_loss_old = allyears[allyears['WinnerHome'] == 0]
home_loss_old['pt_diff'] = home_loss_old['Win_Score'] - home_loss_old['Lose_Score']

c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
print('Point differential in Home Wins and Home Losses before 20-21:', round(home_wins_old['pt_diff'].mean(),2), round(home_loss_old['pt_diff'].mean(),2))

Point differential in Home Wins and Home Losses before 20-21: 13.34 9.68


#### Comparing Close Matchups based on Ken Pom

In [11]:
curr_kp = curr_joined[['Date', 'Winner', 'WinnerScore', 'Loser', 'LoserScore', 'Home', 'Away',
       'WinnerHome','kp_winner', 'kp_rank_winner',
       'kp_loser', 'kp_rank_loser',]]

In [12]:
# Find all games where the Ken Pom difference is less than 3
curr_kp = curr_kp[(abs(curr_kp['kp_winner']-curr_kp['kp_loser'])<3.0) | (abs(curr_kp['kp_rank_winner']-curr_kp['kp_rank_loser'])<25)]

In [13]:
print("2020-21 Season")
print("Number of Games won by home team:",curr_kp['WinnerHome'].sum())
print("Number of Games played:",len(curr_kp))
print("Home Win Percentage:","{:.0%}".format(curr_kp['WinnerHome'].sum()/len(curr_kp)))

2020-21 Season
Number of Games won by home team: 336
Number of Games played: 556
Home Win Percentage: 60%


In [14]:
old_kp = allyears[['Date', 'Win_Score', 'Lose_Score', 'Home', 'Away', 'WinnerHome',
       'KenPomRank_Away', 'KenPomVal_Away','KenPomRank_Home', 'KenPomVal_Home']]
old_kp['kp_rank_winner'] = np.where(old_kp['WinnerHome']==1,old_kp['KenPomRank_Home'],old_kp['KenPomRank_Away'])
old_kp['kp_rank_loser'] = np.where(old_kp['WinnerHome']==0,old_kp['KenPomRank_Home'],old_kp['KenPomRank_Away'])
old_kp['kp_winner'] = np.where(old_kp['WinnerHome']==1,old_kp['KenPomVal_Home'],old_kp['KenPomVal_Away'])
old_kp['kp_loser'] = np.where(old_kp['WinnerHome']==0,old_kp['KenPomVal_Home'],old_kp['KenPomVal_Away'])

c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\kobelb\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWit

In [15]:
old_kp = old_kp[(abs(old_kp['kp_winner']-old_kp['kp_loser'])<3.0) | (abs(old_kp['kp_rank_winner']-old_kp['kp_rank_loser'])<25)]

In [16]:
print("2020-21 Season")
print("Number of Games won by home team:",old_kp['WinnerHome'].sum())
print("Number of Games played:",len(old_kp))
print("Home Win Percentage:","{:.0%}".format(old_kp['WinnerHome'].sum()/len(old_kp)))

2020-21 Season
Number of Games won by home team: 3230
Number of Games played: 5149
Home Win Percentage: 63%


#### Comparing total points per game

In [17]:
print('Average Point total in game in 20-21:',round((curr_season['WinnerScore'] + curr_season['LoserScore']).mean(),2))

Average Point total in game in 20-21: 141.98


In [18]:
print('Average Point total in game prior to 20-21:',round((allyears['Win_Score'] + allyears['Lose_Score']).mean(),2))

Average Point total in game prior to 20-21: 143.9


In [19]:
print('Average number of possessions per team per 40 minutes in 20-21:',round(curr_metrics['pace'].mean(),2))

Average number of possessions per team per 40 minutes in 20-21: 70.33


In [20]:
print('Average number of possessions per team per 40 minutes prior to 20-21:',round(allyears[['Home','Pace_Home']].drop_duplicates()['Pace_Home'].mean(),2))

Average number of possessions per team per 40 minutes prior to 20-21: 69.5


Scoring has gone down 2 total points per game but each team is having on average an additional possession per game. This could be due to teams having sloppier play due to no pre-season and long potential layoffs due to virus spikes

#### Turnovers per game

In [21]:
curr_metrics['tur_perc'].mean()

16.771676300578033

In [22]:
allyears[['Home','Tov_Home']].drop_duplicates()['Tov_Home'].mean()

16.20007501875469

#### Back to Back Same Matchups

In [71]:
curr_season[(curr_season['Winner']==180) | (curr_season['Loser']==180)]

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome
816,2020-12-19,180,94,216,67,180,216,1
867,2020-12-21,180,85,147,77,180,147,1
998,2020-12-27,180,79,213,59,213,180,0
1016,2020-12-28,213,85,180,75,213,180,1
1161,2021-01-02,180,84,126,74,180,126,1
1226,2021-01-03,180,70,126,66,180,126,1
1435,2021-01-09,180,81,332,68,332,180,0
1498,2021-01-10,180,78,332,68,332,180,0
2075,2021-01-26,73,68,180,61,180,73,0
2100,2021-01-27,73,78,180,73,180,73,0


In [72]:
curr_season[(curr_season['Home']==180)]

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome
816,2020-12-19,180,94,216,67,180,216,1
867,2020-12-21,180,85,147,77,180,147,1
1161,2021-01-02,180,84,126,74,180,126,1
1226,2021-01-03,180,70,126,66,180,126,1
2075,2021-01-26,73,68,180,61,180,73,0
2100,2021-01-27,73,78,180,73,180,73,0
2307,2021-01-31,154,72,180,46,180,154,0


In [73]:
b2b = curr_season.sort_values(by=['Home', 'Date'])
b2b['prev_game_date'] = b2b['Date'].shift(1)
b2b['next_game_date'] = b2b['Date'].shift(-1)
b2b['prev_win'] = b2b['WinnerHome'].shift(1)
b2b['prev_game_home'] = b2b['Home'].shift(1)


In [74]:
#All occurences where a team won back to back games
b2b = b2b[b2b['WinnerHome']==b2b['prev_win']]

In [75]:
b2b[(b2b['Home']==180)]

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome,prev_game_date,next_game_date,prev_win,prev_game_home
816,2020-12-19,180,94,216,67,180,216,1,2021-01-30,2020-12-21,1.0,179.0
867,2020-12-21,180,85,147,77,180,147,1,2020-12-19,2021-01-02,1.0,180.0
1161,2021-01-02,180,84,126,74,180,126,1,2020-12-21,2021-01-03,1.0,180.0
1226,2021-01-03,180,70,126,66,180,126,1,2021-01-02,2021-01-26,1.0,180.0
2100,2021-01-27,73,78,180,73,180,73,0,2021-01-26,2021-01-31,0.0,180.0
2307,2021-01-31,154,72,180,46,180,154,0,2021-01-27,2020-12-15,0.0,180.0


In [76]:
b2b = b2b[(b2b['Date'] - b2b['prev_game_date']).dt.days == 1]

In [77]:
b2b[(b2b['Home']==180)]

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome,prev_game_date,next_game_date,prev_win,prev_game_home
1226,2021-01-03,180,70,126,66,180,126,1,2021-01-02,2021-01-26,1.0,180.0
2100,2021-01-27,73,78,180,73,180,73,0,2021-01-26,2021-01-31,0.0,180.0


In [78]:
b2b

,Date,Winner,WinnerScore,Loser,LoserScore,Home,Away,WinnerHome,prev_game_date,next_game_date,prev_win,prev_game_home
304,2020-12-02,4,88,237,71,4,237,1,2020-12-01,2020-12-12,1.0,4.0
848,2020-12-20,314,65,7,60,7,314,0,2020-12-19,2020-12-22,0.0,7.0
1769,2021-01-17,7,83,189,71,7,189,1,2021-01-16,2021-01-16,1.0,7.0
1408,2021-01-09,10,77,101,71,10,101,1,2021-01-08,2021-01-22,1.0,10.0
1961,2021-01-23,10,74,102,61,10,102,1,2021-01-22,2020-11-27,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
994,2020-12-27,353,90,107,77,353,107,1,2020-12-26,2021-01-08,1.0,353.0
2268,2021-01-30,353,86,246,56,353,246,1,2021-01-29,2020-11-28,1.0,353.0
87,2020-11-26,355,51,30,50,355,30,1,2020-11-25,2020-11-27,1.0,355.0
149,2020-11-27,355,76,301,73,355,301,1,2020-11-26,2020-11-30,1.0,355.0


337 occurences of back to back matchups. 201 times, the same team won both. 61.4% of the time